In [26]:
from glob import glob
import json
from pymongo import MongoClient

## Importing the movie data

In [23]:
movies_data = []

for fname in glob("jsonmc-master/movies/*/*.json"):
    with open(fname) as file:
        try:
            contents = json.load(file)
            movies_data.append(contents)
        except UnicodeDecodeError:
            pass

In [24]:
len(movies_data)

341

In [31]:
movies_data[0]

{'name': 'Dr. No',
 'year': 1962,
 'runtime': 110,
 'categories': ['action', 'adventure', 'thriller'],
 'release-date': '1962-10-05',
 'director': 'Terence Young',
 'writer': ['Richard Maibaum',
  'Johanna Harwood',
  'Berkely Mather',
  'Ian Fleming'],
 'actors': ['Sean Connery',
  'Ursula Andress',
  'Joseph Wiseman',
  'Jack Lord',
  'Bernard Lee'],
 'storyline': 'A resourceful British government agent seeks answers in a case involving the disappearance of a colleague and the disruption of the American space program.',
 '_id': ObjectId('5ea58dc1778da3f01ec4f73e')}

## Setting up MongoDB database instance

One can enter data into the mongoDB database like so:

Conncet to the MongoDB client and create a database, then create a document collection in this database and insert our documents (movie entries) into the collection.

In [28]:
client = MongoClient()
db = client.movie_database
movie_collection = db.movies
movie_collection.insert_many(movies_data)

## Basic Commands

We can then count the documents in our collection to check they were inserted correctly. We enter the argument as {} as the first argument is a filter to apply before counting the documents. We enter {} as an empty filter so we retrive a count of all the documents.

In [29]:
movie_collection.count_documents({})

341

We can also retrieve the entire collection of documents (or apply a filter and retrieve matching documents) like so:

In [46]:
all_data = list(movie_collection.find({}))

We can also get all the distinct values in a particular field across all the documents

In [ ]:
names = db.movies.distinct('name')

## Aggregation Requests

We can also make aggregation requests like so, these are a really powerful tool to query, group, and subset the data.

We can use project to get a subset of the fields in each document and to perform operations on the documents to generate new fields for the data.

### Project

In [82]:
pipeline = [
    { "$project": {
        "_id": 0,          # use boolean to say whether to include a field or not - 0 means don't include the field
        "runtime" : 1,     # use 1 to include the field in the projection
        "name": "$name",   # can specify fields by name
        "after1992" : { "$gt": [ "$year", 1992 ] }, # can apply operations such as simple boolean operations
        "when": {"$cond": [
              { "$lt": [ "$year", 1992 ] },
              "Before 1992",
              "After 1992"
            ],
            }, # can also apply more complex conditions to the data using this format - this uses cond's array syntax
        "timeSince1992" : {"$cond" :{
            "if": {"$lt": ["$year", 1992]}, 
            "then": "Before 1992", 
            "else": {"$abs" : {"$subtract": [ 1992, "$year"]}}}},
        # the above line applys the condition operator with the alternative syntax, in both cases we can use field
        # names in the conditional outputs, and even more complex operators within that, here we display "Before 1992"
        # if the date is before 1992, otherwise we subtract the year from 1992 and take the absolute value
        }
    },
]

list(db.movies.aggregate(pipeline))[0:3]

[{'runtime': 110,
  'name': 'Dr. No',
  'after1992': False,
  'when': 'Before 1992',
  'timeSince1992': 'Before 1992'},
 {'runtime': 129,
  'name': 'To Kill a Mockingbird',
  'after1992': False,
  'when': 'Before 1992',
  'timeSince1992': 'Before 1992'},
 {'runtime': 134,
  'name': 'Whatever Happened To Baby Jane?',
  'after1992': False,
  'when': 'Before 1992',
  'timeSince1992': 'Before 1992'}]

### Group

We have to specify/provide a field "_id" which the grouping operation groups documents by, for each document in the group we can perform an operation, one of the most simple is counting the elements in each group.

In [83]:
pipeline = [
    { "$group": {
    "_id": "$year",
    "count": { "$sum": 1 },
    }},
]

list(db.movies.aggregate(pipeline))[0:3]

[{'_id': 1999, 'count': 5},
 {'_id': 1977, 'count': 2},
 {'_id': 1974, 'count': 2}]

We can also group on calculated parameters, such as using a condition to split the data into 2 catgories

In [86]:
pipeline = [
    { "$group": {
    "_id": {"$cond" : {
            "if": {"$lt": ["$year", 1992]}, 
            "then": "Before 1992", 
            "else": "After 1992"}},
    "count": { "$sum": 1 },
  }},
]

list(db.movies.aggregate(pipeline))

[{'_id': 'After 1992', 'count': 289}, {'_id': 'Before 1992', 'count': 52}]

We can also calculate aggregate properties of the group like summing a field, averaging it, and also build an array of values for a field for each document in each group.

In [93]:
pipeline = [
    { "$group": {
    "_id": {"$cond" : {
            "if": {"$lt": ["$year", 1992]}, 
            "then": "Before 1992", 
            "else": "After 1992"}},
    "totalRunTime": { "$sum": "$runtime" },
    "meanRunTime": {"$avg": "$runtime"},
    "maxRunTime": {"$max": "$runtime"},
    "releaseDates": {"$push": "$release-date"},
  }},
]

list(db.movies.aggregate(pipeline))[0:3]

[{'_id': 'After 1992',
  'totalRunTime': 35853,
  'meanRunTime': 124.48958333333333,
  'maxRunTime': 1120,
  'releaseDates': ['1992-12-04',
   '1992-11-20',
   '2093-05-07',
   '1993-06-11',
   '1994-12-16',
   '1994-07-06',
   '1994-10-14',
   '1994-02-04',
   '1994-12-23',
   '1994-06-15',
   '1994-07-29',
   '1993-09-10',
   '1995-11-18',
   '1995-09-15',
   '1995-12-15',
   '1995-10-06',
   '1996-02-16',
   '1996-06-25',
   '1996-05-22',
   '1996-11-15',
   '1996-05-17',
   '1997-11-07',
   '1997-05-09',
   '1997-05-23',
   '1998-07-01',
   '1999-01-10',
   '1999-10-15',
   '1999-05-19',
   '1999-03-31',
   '1999-08-02',
   '2000-12-15',
   '2000-12-20',
   '2000-03-11',
   '2000-05-24',
   '2000-09-29',
   '2000-08-11',
   '2000-07-14',
   '2001-06-26',
   '2001-09-14',
   '2001-11-16',
   '2001-08-24',
   '2001-09-05',
   '2001-12-07',
   '2001-06-22',
   '2001-12-19',
   '2001-07-18',
   '2002-11-15',
   '2002-07-12',
   '2002-05-24',
   '2002-05-16',
   '2002-14-02',
   '2002-1

### Match

One can use match to filter all documents to only include documents that match a condition or set of conditions. We also introduce pipelines with multiple stages here, we can chain together many stages to form complicated and powerful logic. Also depending on the operation being performed operators can look different / have different orders of command, for example these greater than and less than conditions take a different form to the ones we have used above, using the ones above for this operation is not possible/allowed by the syntax.

In [99]:
pipeline = [
    {"$match" : { "$and" : [
        {"year" : {"$lte": 1992}},
        {"runtime": {"$gt": 120}}
    ]}},
    { "$project": {
        "_id": 0,
        "runtime" : 1,
        "year" : 1,
    }},
]

list(db.movies.aggregate(pipeline))[0:3]

[{'year': 1962, 'runtime': 129},
 {'year': 1962, 'runtime': 134},
 {'year': 1965, 'runtime': 172}]

### Unwind

This unwind operation deconstructs a field which has an array of values producing a copy of the document for each value in the array, so we if we have a document with 3 values in the array we will produce 3 documents each identical apart from each having 1 of the entries from the original document's array.

In [101]:
pipeline = [
    { "$unwind" : "$categories"},
]

list(db.movies.aggregate(pipeline))[0:3]

[{'_id': ObjectId('5ea58dc1778da3f01ec4f73e'),
  'name': 'Dr. No',
  'year': 1962,
  'runtime': 110,
  'categories': 'action',
  'release-date': '1962-10-05',
  'director': 'Terence Young',
  'writer': ['Richard Maibaum',
   'Johanna Harwood',
   'Berkely Mather',
   'Ian Fleming'],
  'actors': ['Sean Connery',
   'Ursula Andress',
   'Joseph Wiseman',
   'Jack Lord',
   'Bernard Lee'],
  'storyline': 'A resourceful British government agent seeks answers in a case involving the disappearance of a colleague and the disruption of the American space program.'},
 {'_id': ObjectId('5ea58dc1778da3f01ec4f73e'),
  'name': 'Dr. No',
  'year': 1962,
  'runtime': 110,
  'categories': 'adventure',
  'release-date': '1962-10-05',
  'director': 'Terence Young',
  'writer': ['Richard Maibaum',
   'Johanna Harwood',
   'Berkely Mather',
   'Ian Fleming'],
  'actors': ['Sean Connery',
   'Ursula Andress',
   'Joseph Wiseman',
   'Jack Lord',
   'Bernard Lee'],
  'storyline': 'A resourceful British gove

### An example of combining these together to count the number of films with unique names in each category

In [115]:
pipeline = [
    {"$project" : {
        "_id" : 0,
        "name" : "$name",
        "categories" : 1
    }}, 
    { "$unwind" : "$categories"}, 
    {"$group": {"_id": "$categories",
               "names": {"$addToSet" : "$name" }
               }},
    {"$project" : {
        "_id": 0,
        "category": "$_id",
        "movieCount" : {"$size" : "$names"}
    }},
]
list(db.movies.aggregate(pipeline))[0:5]

# Pipeline stages 
# 1) Subsets data to just names and categories
# 2) unwinds categories (which is an array of strings) so we each document now has a single category
# 3) groups documents by category and adds them to a set of names (a set can only contain each name 
# once so removes any chance of movies being duplicated if they contain the same category twice)
# 4) gets the size of the set - i.e. number of unique names for each category of film

[{'category': 'history', 'movieCount': 6},
 {'category': 'Action', 'movieCount': 7},
 {'category': 'teen', 'movieCount': 1},
 {'category': 'comdey', 'movieCount': 1},
 {'category': 'Police', 'movieCount': 1}]

#### The first projection stage is not required, however, it removes the other unessesary data from the rest of the pipeline processes, for large documents with many fields, or more complicated fields reducing the data to the least required can provide a speed benefit to the overall process. Even for these relatively simple documents a small speed benefit is observed.

In [112]:
%%timeit 

pipeline = [
    {"$project" : {
        "_id" : 0,
        "name" : "$name",
        "categories" : 1
    }}, 
    { "$unwind" : "$categories"}, 
    {"$group": {"_id": "$categories",
               "names": {"$addToSet" : "$name" }
               }},
    {"$project" : {
        "_id": 0,
        "category": "$_id",
        "movieCount" : {"$size" : "$names"}
    }},
]
list(db.movies.aggregate(pipeline))


4.06 ms ± 339 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [114]:
%%timeit 

pipeline = [
    { "$unwind" : "$categories"}, 
    {"$group": {"_id": "$categories",
               "names": {"$addToSet" : "$name" }
               }},
    {"$project" : {
        "_id": 0,
        "category": "$_id",
        "movieCount" : {"$size" : "$names"}
    }},
]
list(db.movies.aggregate(pipeline))


4.34 ms ± 322 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
